# Clickstream Analysis on an Online Retailer Website

The [2000 KDD Cup](http://www.kdd.org/kdd-cup/view/kdd-cup-2000/) featured retail data of an online retailer (now out of business). The original tasks are described in the [organizer's report](https://dl.acm.org/citation.cfm?doid=380995.381033). The original data is available [here](http://www.kdd.org/cupfiles/KDDCup2000.zip) (password=legcare4KDD). Due to time constraints, we investigate only one a small subset. The data is available internally on the Ilias-website of the course.

- `question3agg.data` contains records of 1781 purchases with a special boolean variable indicating whether the purchase was for more than 12 USD 
- `question3agg.names` contains the names of the roughly 500 features that are recorded for each transaction
- faq.html explains some of the fields in more detail; this task is Question 3

Place them in the subfolder `2000-data` relative to this notebook to make the code below work. Then you're already setup

Now, you're setup for this project:
1. Turn the pseudo-code for the A-priori algorithm from the lecture into a quick-and-dirty implementation. (Efficiency is not crucial, because the dataset is not that large. Correctness is. An implementation with python lists should suffice, see the end of the notebook.)
2. Use your algorithm from 1. on the dataset to find frequent itemsets. Note: The algorithm from the lecture runs only on binary features; select only such features or turn a featuere with few (say k) categorical values into k binary features.
Note: This requires the choice of a suitable minimum support; in practice you don't know "suitable" in advance, so binary search the possible values for sigma until the number of frequent itemsets is easy for you to examine by hand, e.g. ~10. 
3. Use the frequent itemsets of 2. to find strong association rules. Note: Analogously to 2., this requires the choice of a proper confidence. What is the best possible confidence that you can achieve?
4. Bonus: Filter your association rules for correlation rules, e.g. using lift as correlation measure.
5. Interpretation: Could the observed rule be an effect of a larger cause?




# 1 Implementation of the Apriori-Algorithm

In [1]:
# Example from the Lecture, slide 8
names = ['milk', 'butter', 'coffee', 'cacao', 'cake', 'sugar', 'tea']
X = [[1, 1, 0, 0, 0, 0, 0],
     [1, 0, 1, 0, 1, 0, 0],
     [1, 0, 0, 1, 1, 0, 0],
     [0, 0, 1, 0, 0, 1, 1],
     [1, 0, 1, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 1, 1]]
# Some 2-itemsets
I1 = [0, 1]
I2 = [1, 5]
I3 = [0, 2]

In [2]:
x = X[1]
any([x[j] == 1 for j in I1])

True

In [3]:
# Return corresponding instance sets
def is_instance(x, I):
    return all([x[j] == 1 for j in I])

def all_instances(X, I):
    return [x for x in X if is_instance(x, I)]

print(all_instances(X, I1))
print(all_instances(X, I2))
print(all_instances(X, I3))

[[1, 1, 0, 0, 0, 0, 0]]
[]
[[1, 0, 1, 0, 1, 0, 0], [1, 0, 1, 0, 0, 1, 0]]


In [4]:
def support(X, I):
    return len(all_instances(X, I))/len(X)
print(support(X, I1))
print(support(X, I2))
print(support(X, I3))


0.16666666666666666
0.0
0.3333333333333333


In [5]:
def is_frequent_itemset(X, L, sigma):
    return all([support(X, I) >= sigma for I in L])

In [6]:
d = {'a': 1, 'b': 1}
for key in d:
    print(key, d[key])

a 1
b 1


In [7]:
def hasAprioriProperty(c, Llast):
    for item in c:
        d = c[:]
        d.remove(item)
        if d not in Llast:
            return False
    return True

def aPriori(X, sigma):
    L1 = [[i] for i in range(len(X[0])) if support(X, [i]) >= sigma]
    L = [L1]
    for k in range(2, len(X) + 1):
        print('start loop with k=%d and L=%s' %(k, L))
        C = [sorted(a + b) for a in L[-1] for b in L1 if b[0] not in a]
        C = [c for i, c in enumerate(C) if c not in C[:i]]    # remove duplicates, because we use lists instead of sets
        print('before Apriori-Check C=%s' %C)
        C = [c for c in C if hasAprioriProperty(c, L[-1])]
        print('after Apriori-Check C=%s' %C)
        Csigma = dict([(tuple(c), 0.0) for c in C])
        for x in X:
            Ct = [c for c in C if is_instance(x, c)]
            for c in Ct:
                Csigma[tuple(c)] += 1/len(X)
        C = [c for c in C if Csigma[tuple(c)] >= sigma]
        if len(C) == 0:
            break
        L += [C]       
    return [c for Lk in L for c in Lk ]

print(aPriori(X, 0.3))
print(aPriori(X, 0.5))
print(aPriori(X, 0.7))    

start loop with k=2 and L=[[[0], [2], [4], [5], [6]]]
before Apriori-Check C=[[0, 2], [0, 4], [0, 5], [0, 6], [2, 4], [2, 5], [2, 6], [4, 5], [4, 6], [5, 6]]
after Apriori-Check C=[[0, 2], [0, 4], [0, 5], [0, 6], [2, 4], [2, 5], [2, 6], [4, 5], [4, 6], [5, 6]]
start loop with k=3 and L=[[[0], [2], [4], [5], [6]], [[0, 2], [0, 4], [2, 5], [5, 6]]]
before Apriori-Check C=[[0, 2, 4], [0, 2, 5], [0, 2, 6], [0, 4, 5], [0, 4, 6], [2, 4, 5], [2, 5, 6], [0, 5, 6], [4, 5, 6]]
after Apriori-Check C=[]
[[0], [2], [4], [5], [6], [0, 2], [0, 4], [2, 5], [5, 6]]
start loop with k=2 and L=[[[0], [2], [5]]]
before Apriori-Check C=[[0, 2], [0, 5], [2, 5]]
after Apriori-Check C=[[0, 2], [0, 5], [2, 5]]
[[0], [2], [5]]
start loop with k=2 and L=[[]]
before Apriori-Check C=[]
after Apriori-Check C=[]
[]


In [8]:
# Example from Lecture p. 21
names = ['I1', 'I2', 'I3', 'I4', 'I5']
X     = [[1,    1,    0,    0,    1],
         [0,    1,    0,    1,    0],
         [0,    1,    1,    0,    0],
         [1,    1,    0,    1,    0],
         [1,    0,    1,    0,    0],
         [0,    1,    1,    0,    0],
         [1,    0,    1,    0,    0],
         [1,    1,    1,    0,    1],
         [1,    1,    1,    0,    0]]

print(aPriori(X, 2/9))

start loop with k=2 and L=[[[0], [1], [2], [3], [4]]]
before Apriori-Check C=[[0, 1], [0, 2], [0, 3], [0, 4], [1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4]]
after Apriori-Check C=[[0, 1], [0, 2], [0, 3], [0, 4], [1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4]]
start loop with k=3 and L=[[[0], [1], [2], [3], [4]], [[0, 1], [0, 2], [0, 4], [1, 2], [1, 3], [1, 4]]]
before Apriori-Check C=[[0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 2, 3], [0, 2, 4], [0, 3, 4], [1, 2, 3], [1, 2, 4], [1, 3, 4]]
after Apriori-Check C=[[0, 1, 2], [0, 1, 4]]
start loop with k=4 and L=[[[0], [1], [2], [3], [4]], [[0, 1], [0, 2], [0, 4], [1, 2], [1, 3], [1, 4]], [[0, 1, 2], [0, 1, 4]]]
before Apriori-Check C=[[0, 1, 2, 3], [0, 1, 2, 4], [0, 1, 3, 4]]
after Apriori-Check C=[]
[[0], [1], [2], [3], [4], [0, 1], [0, 2], [0, 4], [1, 2], [1, 3], [1, 4], [0, 1, 2], [0, 1, 4]]


# 2 Frequent itemsets in the KDD 2000 data

In [9]:
# load dataset and check top rows
import pandas as pd
with open('dataset/question3agg.names', 'r') as header:
    header_row = header.readlines()[8:]
df = pd.read_csv('dataset/question3agg.data', names=header_row)
pd.set_option('display.max_columns', 500)
df.head()

/root/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4,12,126,132,139,206,228,261,272,284) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,"WhichDoYouWearMostFrequent: trouser socks,hosiery,casual socks,athletic socks.","YourFavoriteLegcareBrand: eShave,Lucky Chick,VeinGard,Tweezerman,Swiss Balance,Ovacion,Covermark,Ya Babi,Medea,Dr\. Perricone,TendSkin,IColoniali,Dr\. Schon,Nature Made,Bio Depiless,Conair,Natural Line,Kleb Sole,Living Earth,Cellasene,DailyHerbs,Kneipp,Juvena,Epilady.","Registration Gender: Female,Male.","NumberOfChildren: [ordered] 0, 1, 2, 3, 4 or more.","DoYouPurchaseForOthers: NULL,False.","HowDoYouDressForWork: business dress,business casual,comfortable / athletic,very casual.","HowManyPairsDoYouPurchase: [ordered] 1 to 5, 6 to 10, 11 to 15, 15 or more.","YourFavoriteLegwearBrand: Ellen Tracy,Oroblu,Belly Basics,American Essentials,Danskin,Hot Sox,Donna Karan,Evan Picone,Givenchy,Greg Norman,DKNY,Hanes,Round the Clock,Falke,Berkshire.","WhoMakesPurchasesForYou: parent,spouse,siblings,friend.","NumberOfAdults: [ordered] 0, 1, 2, 3 or more.","HowDidYouHearAboutUs: in the news,e-mail,print ad,direct mail,other,friend / family.","Company: COMPANY 2605,COMPANY 416,COMPANY 2975,COMPANY 3051,COMPANY 780,COMPANY 3055,COMPANY 1608,COMPANY 3059,COMPANY 2575,COMPANY 2330,COMPANY 1169,COMPANY 1207,COMPANY 1209,COMPANY 2982,COMPANY 1571,COMPANY 1733,COMPANY 1737,COMPANY 1455,COMPANY 796,COMPANY 673,COMPANY 713,COMPANY 395,COMPANY 2183,COMPANY 2870,COMPANY 3070,COMPANY 1587,COMPANY 1626,COMPANY 842,COMPANY 2479,COMPANY 445,COMPANY 2235,COMPANY 2882,COMPANY 2922,COMPANY 3089,COMPANY 613,COMPANY 452,COMPANY 615,COMPANY 617,COMPANY 1085,COMPANY 1086,COMPANY 2092,COMPANY 503,COMPANY 1932,COMPANY 2258,COMPANY 1935,COMPANY 1658,COMPANY 594,COMPANY 1258,COMPANY 1099,COMPANY 2262,COMPANY 1944,COMPANY 519,COMPANY 920,COMPANY 3032,COMPANY 3036,COMPANY 2678,COMPANY 482,COMPANY 2273,COMPANY 1792,COMPANY 402,COMPANY 409,NULL,COMPANY 2963,COMPANY 2682,COMPANY 1273,COMPANY 933.","SendEmail: NULL,True,False.","HowOftenDoYouPurchase: [ordered] once a year, every 6 months, each week.","HowDidYouFindUs: Web/Banner Ad,News Story,Search Engine,Friend/Co-worker,Magazine Ad,Other.",City: ignore.,"Country: United States,NULL.","US State: IA,ID,IL,IN,AK,AL,AR,RI,AZ,SC,CA,KS,CO,KY,CT,LA,TN,DC,DE,TX,MA,MD,ME,MI,UT,MN,MO,MS,MT,VA,NC,ND,NE,NH,NJ,VT,NM,FL,NV,WA,NY,WI,OH,GA,OK,WV,WY,OR,PA,HI.",Account Creation Date: date.,Account Creation Date_Time: time.,Year of Birth: continuous.,"Email: NET,GOV,COM,EDU,ORG,Gazelle,Other,NULL.",Login Failure Count: continuous.,Customer ID: continuous.,"Truck Owner: NULL,True,False.","RV Owner: NULL,True,False.","Motorcycle Owner: NULL,True,False.",Value Of All Vehicles: continuous.,Age: continuous.,Other Indiv\. Age: continuous.,"Marital Status: Inferred Married,Single,Inferred Single,Married,NULL.","Working Woman: NULL,True,False.","Mail Responder: NULL,True,False.","Bank Card Holder: NULL,True,False.","Gas Card Holder: NULL,True,False.","Upscale Card Holder: NULL,True,False.","Unknown Card Type: NULL,True,False.","TE Card Holder: NULL,True,False.","Premium Card Holder: NULL,True,False.","Presence Of Children: NULL,True,False.",Number Of Adults: continuous.,"Estimated Income Code: [ordered] Under $15;000, $15;000-$19;999, $20;000-$29;999, $30;000-$39;999, $40;000-$49;999, $50;000-$74;999, $75;000-$99;999, $100;000-$124;999, $125;000 OR MORE.","Home Market Value: [ordered] $1;000-$24;999, $25;000-$49;999, $50;000-$74;999, $75;000-$99;999, $100;000-$124;999, $125;000-$149;999, $150;000-$174;999, $175;000-$199;999, $200;000-$224;999, $225;000-$249;999, $250;000-$274;999, $275;000-$299;999, $300;000-$349;999, $350;000-$399;999, $400;000-$449;999, $450;000-$499;999, $500;000-$774;999, $775;000-$999;999, $1;000;000+.","New Car Buyer: NULL,True.","Vehicle Lifestyle: FULL SIZE (STANDARD/LUXURY),IMPORT (STANDARD/ECONOMY),SPECIALTY (MIDSIZE/SMALL),PERSONAL LUXURY CAR,STATION WAGON,REGULAR (MIDSIZE/SMALL),TRUCK OR UTILITY VEHICLE,NULL.","Property Type: apartment(5+ units),2-4 unit(duplex;triplex;quad),mobile_home,misc\. residential (condo store/flat),single f

In [10]:
# basic metrics for numerical columns
df.describe()

,Customer ID: continuous.,Order Amount Sum Percent Submitted: continuous.,Order Amount Sum Percent Complete: continuous.,Order Amount Sum Percent In process: continuous.,Order Amount Sum Percent Using VISA: continuous.,Order Amount Sum Percent Using MC: continuous.,Order Amount Sum Percent Using AMEX: continuous.,Order Amount Sum Percent On Thursday: continuous.,Order Amount Sum Percent On Saturday: continuous.,Order Amount Sum Percent On Tuesday: continuous.,Order Amount Sum Percent On Sunday: continuous.,Order Amount Sum Percent On Wednesday: continuous.,Order Amount Sum Percent On Friday: continuous.,Order Amount Sum Percent On Monday: continuous.,Percent Of Products Purchased On Thursday: continuous.,Percent Of Products Purchased On Saturday: continuous.,Percent Of Products Purchased On Tuesday: continuous.,Percent Of Products Purchased On Sunday: continuous.,Percent Of Products Purchased On Wednesday: continuous.,Percent Of Products Purchased On Friday: continuous.,Percent Of Products Purchased On Monday: continuous.,Order Amount Sum Percent During 6pm-12pm: continuous.,Order Amount Sum Percent During 3am-7am: continuous.,Order Amount Sum Percent During 12am-3am: continuous.,Order Amount Sum Percent During 3pm-6pm: continuous.,Order Amount Sum Percent During 11am-3pm: continuous.,Order Amount Sum Percent During 7am-11am: continuous.,Percentage Of Products Purchased At Local Hour of Day Bin 6pm-12pm: continuous.,Percentage Of Products Purchased At Local Hour of Day Bin 3am-7am: continuous.,Percentage Of Products Purchased At Local Hour of Day Bin 12am-3am: continuous.,Percentage Of Products Purchased At Local Hour of Day Bin 3pm-6pm: continuous.,Percentage Of Products Purchased At Local Hour of Day Bin 11am-3pm: continuous.,Percentage Of Products Purchased At Local Hour of Day Bin 7am-11am: continuous.,Order Line Amount Sum Percent AmericanEssentials: continuous.,Order Line Amount Sum Percent Hanes: continuous.,Order Line Amount Sum Percent HotSox: continuous.,Order Line Amount Sum Percent EllenTracy: continuous.,Order Line Amount Sum Percent Danskin: continuous.,Order Line Amount Sum Percent DKNY: continuous.,Order Line Amount Sum Percent NicoleMiller: continuous.,Order Line Amount Sum Percent Berkshire: continuous.,Order Line Amount Sum Percent DonnaKaran: continuous.,Order Line Amount Sum Percent Oroblu: continuous.,Order Line Amount Sum Percent Legwear: continuous.,Percentage Legwear_size 3x-4x Purchased: continuous.,Percentage Legwear_size Q-Petite Purchased: continuous.,Percentage Legwear_size 1/2 Purchased: continuous.,Percentage Legwear_size AB Purchased: continuous.,Percentage Legwear_size 5x-6x Purchased: continuous.,Percentage Legwear_size 1 Purchased: continuous.,Percentage Legwear_size 3 Purchased: continuous.,Percentage Legwear_size 2+ Purchased: continuous.,Percentage Legwear_size 4 Purchased: continuous.,Percentage Legwear_size 5 Purchased: continuous.,Percentage Legwear_size 1P Purchased: continuous.,Percentage Legwear_size A Purchased: continuous.,Percentage Legwear_size 1X Purchased: continuous.,Percentage Legwear_size B Purchased: continuous.,Percentage Legwear_size C Purchased: continuous.,Percentage Legwear_size D Purchased: continuous.,Percentage Legwear_size E Purchased: continuous.,Percentage Legwear_size F Purchased: continuous.,Percentage Legwear_size I Purchased: continuous.,Percentage Legwear_size L Purchased: continuous.,Percentage Legwear_size M Purchased: continuous.,Percentage Legwear_size P Purchased: continuous.,Percentage Legwear_size S Purchased: continuous.,Percentage Legwear_size T Purchased: continuous.,Percentage Legwear_size S/M Purchased: continuous.,Percentage Legwear_size 2P Purchased: continuous.,Percentage Legwear_size CD Purchased: continuous.,Percentage Legwear_size 2X Purchased: continuous.,Percentage Legwear_size L/XL Purchased: continuous.,Percentage Legwear_size 3P Purchased: continuous.,Percentage Legwear_size C/D Purchased: continuous.,Percentage Legwear_size 3X Purcha

In [11]:
# Select some of the colums by name. Hint: Use Tab-completion for the names
df_selection = df['Registration Gender: Female,Male.\n']
print(df_selection.head())
df_selection = df[['Spend over $12 per order on average: True,False.\n', 'Working Woman: NULL,True,False.\n', 'Unknown Card Type: NULL,True,False.\n', 'Truck Owner: NULL,True,False.\n']]
print(df_selection.head())

0      Male
1      Male
2      Male
3      Male
4    Female
Name: Registration Gender: Female,Male.\n, dtype: object
  Spend over $12 per order on average: True,False.\n  \
0                                              False   
1                                               True   
2                                              False   
3                                               True   
4                                               True   

  Working Woman: NULL,True,False.\n Unknown Card Type: NULL,True,False.\n  \
0                             False                                 False   
1                               NaN                                   NaN   
2                             False                                 False   
3                               NaN                                   NaN   
4                             False                                  True   

  Truck Owner: NULL,True,False.\n  
0                            True  
1          

In [12]:
# If you are new to python, then working with the pandas-dataframe may be overwhelming.
# Working with the data as list of lists may be cleaner -- and sufficient
data_selection_as_list = df_selection.values.tolist()
len(data_selection_as_list)

1781

In [13]:
# translate True/False to 1/0 and filter nan 
from math import isnan
X = []
for x in data_selection_as_list:
    if True in [isnan(i) for i in x]:
        continue
    X.append([int(i) for i in x])
len(X)

1419

In [14]:
print(aPriori(X,.1))

start loop with k=2 and L=[[[0], [1], [2], [3]]]
before Apriori-Check C=[[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]]
after Apriori-Check C=[[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]]
start loop with k=3 and L=[[[0], [1], [2], [3]], [[1, 2], [2, 3]]]
before Apriori-Check C=[[0, 1, 2], [1, 2, 3], [0, 2, 3]]
after Apriori-Check C=[]
[[0], [1], [2], [3], [1, 2], [2, 3]]
